In [2]:
import json
import pinecone
import pandas as pd
import streamlit as st
from datetime import datetime
from google.oauth2 import service_account
from google.cloud import storage
from typing import List, Dict, Union, Tuple, Optional

/Users/artyod/Desktop/VS Code/fo_ai/venv/lib/python3.10/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
bucket_name = "bucket_g_cloud_service_1"

# Create API client for Google Cloud Storage

credentials = service_account.Credentials.from_service_account_info(st.secrets["gcp_service_account"])
client = storage.Client(credentials=credentials)

In [4]:
# Import necessary libraries
import pandas as pd

file_path = "fo/df_requests.json"
# Define the file path
bucket = client.bucket(bucket_name)
blob = bucket.blob(file_path)

# Download content from the blob
content = blob.download_as_text()

print()
# Convert the JSON string to a Python dictionary
content_dict = json.loads(content)

df = pd.read_json(content, orient="split")

df

,date,question,vehicle_flag,vehicle,rim,possible_rim_id,rim_entry_id,rim_details_index,final_response,correct_response,comment,tag,helpful,ticket,specific_context,general_context,tokens_used,model_parameters


In [5]:
file_path = "fo/df_rim_details.json"
# Define the file path
bucket = client.bucket(bucket_name)
blob = bucket.blob(file_path)

# Download content from the blob
content = blob.download_as_text()

print()
# Convert the JSON string to a Python dictionary
content_dict = json.loads(content)

df = pd.read_json(content, orient="split")

df

,requests_index,vehicle_flag,vehicle,hersteller,design,farbe,lochkreise,zoll,breite,einpresstiefe,nabe,traglast,gewicht,hersteller_design_mapped,rim_details_system,mapping_information


In [14]:
file_path = "fo/df_dynamic_contexts.json"
# Define the file path
bucket = client.bucket(bucket_name)
blob = bucket.blob(file_path)

# Download content from the blob
content = blob.download_as_text()

print()
# Convert the JSON string to a Python dictionary
content_dict = json.loads(content)

df = pd.read_json(content, orient="split")

df

,rim_details_index,context,n_tokens,emb context
0,0,"Fahrzeug-Informationen : {'trade': 'Seat', 'mo...",173,"[-0.0073924419, 0.0329701528, 0.0184436999, -0..."
1,0,Felgen-Details und Beschreibung : {'rim_detail...,755,"[0.0088257091, 0.030680488800000003, -0.010001..."
2,0,Gutachtenerklaerung zur Passgenauigkeit : ['Pa...,416,"[0.0028126917, 0.0146326544, 0.0075093876, -0...."
3,0,Anforderungen an das Felgen-Gutachten : [{'ico...,513,"[0.005831548000000001, 0.0233810451, 0.0083102..."
4,0,"Alternative Felgengrößen : [{'inch': 18, 'link...",206,"[0.0230779275, 0.028926538300000002, 0.0114839..."
5,0,Alternative Felgenfarben : [{'name': 'R³ Wheel...,360,"[0.0162648335, 0.0271908268, -0.0037627083, -0..."
6,0,Informationen zum Felgen-Gutachten : {'419203'...,104,"[-0.0136326794, 0.034177903100000004, 0.007030..."
7,0,Informationen zur Reifendimension 225/35 R19 :...,1113,"[0.017395094, 0.0265554208, 0.0084661571, -0.0..."
8,0,Informationen zur Reifendimension 235/30 R19 :...,903,"[0.0139274616, 0.0321048945, 0.007383314, -0.0..."
9,0,Informationen zur Reifendimension 235/35 R19 :...,889,"[0.0156690571, 0.0346454233, 0.0085664745, -0...."


In [47]:
filepath = "data/processed/faq_after_emb.pkl"
df1 = pd.read_pickle(filepath)

filepath = "data/processed/template_after_emb.pkl"
df2 = pd.read_pickle(filepath)

In [48]:
df = pd.concat([df1, df2], axis=0)

In [49]:
df["id"] = [str(i) for i in range(len(df))]

In [50]:
df["url"] = df["url"].fillna("")

In [51]:
df = df[["id", "category", "topic", "question", "answer", "url", "content", "n_tokens", "vector_ada", "local"]]

In [56]:
df.head()

,id,category,topic,question,answer,url,content,n_tokens,vector_ada,local
0,0,FAQ2020,Kaufabwicklung,Was ist der Felgenberater und wie verwende ich...,Unser virtueller Felgenberater wurde entwickel...,https://www.felgenoutlet.de/de/felgenberater,Topic: Kaufabwicklung [SEP] Question: Was ist ...,144,"[0.008489251136779785, 0.03744645044207573, 0....",NaN
1,1,FAQ2020,Kaufabwicklung,Welche Reifen sind auf den Felgen?,Haben Sie Ihre gewünschte Felge aus unserem So...,,Topic: Kaufabwicklung [SEP] Question: Welche R...,92,"[0.020079495385289192, 0.03007490187883377, 0....",NaN
2,2,FAQ2020,Kaufabwicklung,Welches Zubehör wird mitgeliefert?,Im Warenkorb haben Sie außerdem die Möglichkei...,,Topic: Kaufabwicklung [SEP] Question: Welches ...,77,"[0.02538178488612175, 0.027474189177155495, 0....",NaN
3,3,FAQ2020,Kaufabwicklung,Welches Zubehör wird mitgeliefert?,"Sofern laut Gutachten vorgeschrieben, sind bei...",https://www.felgenoutlet.de/de/pflegeprodukte-...,Topic: Kaufabwicklung [SEP] Question: Welches ...,121,"[0.019405554980039597, 0.03225148469209671, 0....",NaN
4,4,FAQ2020,Kaufabwicklung,Wie kann ich Reifen bestellen?,"Benötigen Sie neue Reifen, werden Ihnen in uns...",https://www.felgenoutlet.de/de/blog/reifenbeze...,Topic: Kaufabwicklung [SEP] Question: Wie kann...,89,"[0.037569303065538406, 0.025705311447381973, 0...",NaN


In [53]:
df.shape

(131, 10)

In [54]:
if not index_name in pinecone.list_indexes():
    pinecone.create_index(index_name, dimension=len(df["vector_ada"].tolist()[0]), metric="cosine")

index = pinecone.Index(index_name)

In [55]:
from tqdm.auto import tqdm

# from search_options_bricks import instructions

batch_size = 32

for i in tqdm(range(0, len(df), batch_size)):
    i_end = min(i + batch_size, len(df))
    df_slice = df.iloc[i:i_end]
    to_upsert = [
        (
            row["id"],
            row["vector_ada"],
            {
                "category": row["category"],
                "topic": row["topic"],
                "question": row["question"],
                "url": row["url"],
                "content": row["content"],
                "n_tokens": row["n_tokens"],
            },
        )
        for _, row in df_slice.iterrows()
    ]
    index.upsert(vectors=to_upsert)

  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 5/5 [00:05<00:00,  1.07s/it]


In [57]:
df_all_rims = pd.read_pickle("data/df_all_rims.pkl")


file_path = "fo/df_all_rims.json"
# Define the file path
# bucket = client.bucket(bucket_name)
# blob = bucket.blob(file_path)
# json_content = df_all_rims.to_json(orient="split")
# blob.upload_from_string(json_content, content_type="application/json")
print(df_all_rims.shape)
df_all_rims.head()

(48500, 17)


,felgen_id,felgen_anwendungs_id,hersteller,design,farbe,lochkreise,zoll,breite,einpresstiefe,nabe,traglast,gewicht,lieferbar,hd combined,hd emb combined,vr combined,vr emb combined
0,17,13,TOMASON,TN1,hyperblack poliert,4/100,17.0,8.0,35.0,63.4,580,9825,True,Hersteller: TOMASON; Design: TN1,"[0.010997028090059757, 0.0013537120539695024, ...",Lochkreise: 4/100; Einpresstiefe: 35.0; Nabe: ...,"[0.004250480327755213, 0.010106037370860577, 0..."
1,17,14,TOMASON,TN1,hyperblack poliert,5/100,18.0,8.5,30.0,63.4,580,11370,False,Hersteller: TOMASON; Design: TN1,"[0.010997028090059757, 0.0013537120539695024, ...",Lochkreise: 5/100; Einpresstiefe: 30.0; Nabe: ...,"[-0.0011827027192339301, 0.005210462491959333,..."
2,17,15,TOMASON,TN1,hyperblack poliert,5/108,18.0,8.5,40.0,72.6,625,11370,False,Hersteller: TOMASON; Design: TN1,"[0.010997028090059757, 0.0013537120539695024, ...",Lochkreise: 5/108; Einpresstiefe: 40.0; Nabe: ...,"[-0.003893747692927718, 0.006592674646526575, ..."
3,17,16,TOMASON,TN1,hyperblack poliert,5/112,18.0,8.5,45.0,72.6,650,11370,False,Hersteller: TOMASON; Design: TN1,"[0.010997028090059757, 0.0013537120539695024, ...",Lochkreise: 5/112; Einpresstiefe: 45.0; Nabe: ...,"[-0.005662090145051479, 0.011956925503909588, ..."
4,21,25,TOMASON,TN4,hyperblack poliert,5/112,18.0,8.5,30.0,72.6,750,13455,False,Hersteller: TOMASON; Design: TN4,"[0.0006953853880986571, -0.0003292182809673249...",Lochkreise: 5/112; Einpresstiefe: 30.0; Nabe: ...,"[-0.004306187387555838, 0.015162999741733074, ..."


In [58]:
columns_to_keep = ["felgen_id", "hersteller", "design", "hd combined", "hd emb combined"]

# Drop duplicate rows based on "hd combined" column
df_hd = df_all_rims.drop_duplicates(subset=["hd combined"])

# Keep only the specified columns
df_hd = df_hd[columns_to_keep]

print(df_hd.shape)
df_hd.head()

(739, 5)


,felgen_id,hersteller,design,hd combined,hd emb combined
0,17,TOMASON,TN1,Hersteller: TOMASON; Design: TN1,"[0.010997028090059757, 0.0013537120539695024, ..."
4,21,TOMASON,TN4,Hersteller: TOMASON; Design: TN4,"[0.0006953853880986571, -0.0003292182809673249..."
14,161,Keskin,KT10,Hersteller: Keskin; Design: KT10,"[0.01046041026711464, -0.0037310803309082985, ..."
15,1181,RC-Design,Matrix T,Hersteller: RC-Design; Design: Matrix T,"[-0.0008942902786657214, 0.018468448892235756,..."
20,1156,RC-Design,RC 14,Hersteller: RC-Design; Design: RC 14,"[-0.008932876400649548, 0.013271701522171497, ..."


In [62]:
df_hd

,felgen_id,hersteller,design,hd combined,hd emb combined,id
0,17,TOMASON,TN1,Hersteller: TOMASON; Design: TN1,"[0.010997028090059757, 0.0013537120539695024, ...",hd-0
1,21,TOMASON,TN4,Hersteller: TOMASON; Design: TN4,"[0.0006953853880986571, -0.0003292182809673249...",hd-1
2,161,Keskin,KT10,Hersteller: Keskin; Design: KT10,"[0.01046041026711464, -0.0037310803309082985, ...",hd-2
3,1181,RC-Design,Matrix T,Hersteller: RC-Design; Design: Matrix T,"[-0.0008942902786657214, 0.018468448892235756,...",hd-3
4,1156,RC-Design,RC 14,Hersteller: RC-Design; Design: RC 14,"[-0.008932876400649548, 0.013271701522171497, ...",hd-4
...,...,...,...,...,...,...
734,10342,ProLine,CX300,Hersteller: ProLine; Design: CX300,"[0.010348143056035042, 0.01238570548593998, -0...",hd-734
735,21398,TOMASON,TN29 Superlight,Hersteller: TOMASON; Design: TN29 Superlight,"[0.01611938700079918, 0.008888503536581993, 0....",hd-735
736,11152,Momo,Quantum Evo,Hersteller: Momo; Design: Quantum Evo,"[0.0016250525368377566, 0.011521449312567711, ...",hd-736
737,11170,Momo,Stealth,Hersteller: Momo; Design: Stealth,"[-0.0008462950354442, -0.0012312026228755713, ...",hd-737


In [33]:
print(type(df_hd))

<class 'pandas.core.frame.DataFrame'>


In [25]:
print(set(df_hd["hd combined"].apply(type)))

{<class 'str'>}


In [8]:
import pinecone
from tqdm.autonotebook import tqdm

pinecone_api_key = st.secrets["PINECONE_API_KEY"]
pinecone_environment = st.secrets["PINECONE_environment"]


pinecone.init(api_key=pinecone_api_key, environment=pinecone_environment)
index_name = "fo"

In [63]:
print(df_hd.dtypes)

0
felgen_id          object
hersteller         object
design             object
hd combined        object
hd emb combined    object
id                 object
dtype: object


In [42]:
pinecone.delete_index("fo")

In [64]:
from tqdm.auto import tqdm

# from search_options_bricks import instructions

batch_size = 32
categories_set = set()

for i in tqdm(range(0, len(df_hd), batch_size)):
    i_end = min(i + batch_size, len(df_hd))
    df_slice = df_hd.iloc[i:i_end]
    to_upsert = [
        (
            row["id"],
            row["hd emb combined"],
            {
                "category": "hersteller design",
                "felgen_id": row["felgen_id"],
                "hersteller": row["hersteller"],
                "design": row["design"],
                "hd combined": row["hd combined"],
            },
        )
        for _, row in df_slice.iterrows()
    ]
    index.upsert(vectors=to_upsert)

  0%|          | 0/24 [00:00<?, ?it/s]

100%|██████████| 24/24 [00:28<00:00,  1.20s/it]


In [82]:
def map_similar_rims_pinecone(
    question: str,
    index,
    filter_query,
    engine_name: str = "text-embedding-ada-002",
    limit: int = 5,
    threshold: float = 0.95,
    max_len: int = 1200,
    context_fields: List[str] = ["hersteller", "design"],
    detail_fields: List[str] = ["similarity"],
) -> Tuple[List[Dict[str, Union[str, float]]], List[Dict[str, float]]]:
    """
    Create a context for a question by finding the most similar rims from a Pinecone index.

    Parameters:
        question (str): The question to map.
        index: Pinecone index.
        ...

    Returns:
        Tuple[List[Dict[str, Union[str, float]]], List[Dict[str, float]]]: Returns contexts and their details.
    """
    # Get the embeddings for the question
    q_embed = openai.Embedding.create(input=question, engine=engine_name)["data"][0]["embedding"]

    # Query Pinecone index
    res = index.query(q_embed, filter=filter_query, top_k=limit, include_metadata=True)
    # res = index.query(q_embed, top_k=limit, include_metadata=True)

    contexts, context_details = [], []

    for match in sorted(res["matches"], key=lambda x: x["score"], reverse=True):
        # Check for threshold
        if match["score"] < threshold:
            break

        # Extract metadata
        metadata = match["metadata"]

        context_data = {field: metadata.get(field, "") for field in context_fields}
        contexts.append(context_data)

        detail_data = {field: match["score"] for field in detail_fields}
        context_details.append(detail_data)

    return contexts, context_details

In [109]:
import openai

question = f"hersteller: MAM, design: GT1 silver painted"

contexts, context_details = map_similar_rims_pinecone(
    question=question,
    index=index,
    filter_query={"category": "hersteller design"},
    engine_name="text-embedding-ada-002",  # You can replace this with your own engine name
    max_len=1200,
    limit=1,
    threshold=0.85,
    context_fields=["hersteller", "design"],
    detail_fields=["similarity"],
)

In [110]:
print(contexts)
print(context_details)

[{'hersteller': 'MAM', 'design': 'GT1'}]
[{'similarity': 0.956620574}]


In [73]:
df_all_rims["id"] = "vr-" + df_all_rims.index.astype(str)
print(df_all_rims.shape)
df_all_rims.head()

(48500, 18)


,felgen_id,felgen_anwendungs_id,hersteller,design,farbe,lochkreise,zoll,breite,einpresstiefe,nabe,traglast,gewicht,lieferbar,hd combined,hd emb combined,vr combined,vr emb combined,id
0,17,13,TOMASON,TN1,hyperblack poliert,4/100,17.0,8.0,35.0,63.4,580,9825,True,Hersteller: TOMASON; Design: TN1,"[0.010997028090059757, 0.0013537120539695024, ...",Lochkreise: 4/100; Einpresstiefe: 35.0; Nabe: ...,"[0.004250480327755213, 0.010106037370860577, 0...",vr-0
1,17,14,TOMASON,TN1,hyperblack poliert,5/100,18.0,8.5,30.0,63.4,580,11370,False,Hersteller: TOMASON; Design: TN1,"[0.010997028090059757, 0.0013537120539695024, ...",Lochkreise: 5/100; Einpresstiefe: 30.0; Nabe: ...,"[-0.0011827027192339301, 0.005210462491959333,...",vr-1
2,17,15,TOMASON,TN1,hyperblack poliert,5/108,18.0,8.5,40.0,72.6,625,11370,False,Hersteller: TOMASON; Design: TN1,"[0.010997028090059757, 0.0013537120539695024, ...",Lochkreise: 5/108; Einpresstiefe: 40.0; Nabe: ...,"[-0.003893747692927718, 0.006592674646526575, ...",vr-2
3,17,16,TOMASON,TN1,hyperblack poliert,5/112,18.0,8.5,45.0,72.6,650,11370,False,Hersteller: TOMASON; Design: TN1,"[0.010997028090059757, 0.0013537120539695024, ...",Lochkreise: 5/112; Einpresstiefe: 45.0; Nabe: ...,"[-0.005662090145051479, 0.011956925503909588, ...",vr-3
4,21,25,TOMASON,TN4,hyperblack poliert,5/112,18.0,8.5,30.0,72.6,750,13455,False,Hersteller: TOMASON; Design: TN4,"[0.0006953853880986571, -0.0003292182809673249...",Lochkreise: 5/112; Einpresstiefe: 30.0; Nabe: ...,"[-0.004306187387555838, 0.015162999741733074, ...",vr-4


In [75]:
from tqdm.auto import tqdm

# from search_options_bricks import instructions

batch_size = 32
categories_set = set()

for i in tqdm(range(0, len(df_all_rims), batch_size)):
    i_end = min(i + batch_size, len(df_all_rims))
    df_slice = df_all_rims.iloc[i:i_end]
    to_upsert = [
        (
            row["id"],
            row["vr emb combined"],
            {
                "category": "vehicle rim",
                "felgen_id": row["felgen_id"],
                "felgen_anwendungs_id": row["felgen_anwendungs_id"],
                "hersteller": row["hersteller"],
                "design": row["design"],
                "farbe": row["farbe"],
                "lochkreise": row["lochkreise"],
                "zoll": row["zoll"],
                "breite": row["breite"],
                "einpresstiefe": row["einpresstiefe"],
                "nabe": row["nabe"],
                "traglast": row["traglast"],
                "gewicht": row["gewicht"],
                "vr combined": row["vr combined"],
            },
        )
        for _, row in df_slice.iterrows()
    ]
    index.upsert(vectors=to_upsert)

100%|██████████| 1516/1516 [30:15<00:00,  1.20s/it]


In [105]:
from typing import List, Dict


def create_filter_query_vr(hd: List[Dict[str, str]], zoll: Optional[str] = None) -> Dict:
    """
    Create a MongoDB filter query based on the input list of dictionaries containing 'hersteller', 'design', and 'zoll'.

    Parameters:
        hd (List[Dict[str, str]]): A list of dictionaries, each containing 'hersteller' and 'design' fields.
        zoll (Optional[str]): The zoll value as a string. Default is None.

    Returns:
        Dict: A MongoDB filter query.
    """

    filter_query = {"$and": [{"category": "vehicle rim"}]}

    for item in hd:
        hersteller_value = item.get("hersteller", "")
        design_value = item.get("design", "")

        if hersteller_value:
            filter_query["$and"].append({"hersteller": hersteller_value})

        if design_value:
            filter_query["$and"].append({"design": design_value})

    if zoll:
        filter_query["$and"].append({"zoll": zoll})

    return filter_query

In [106]:
filter_query = create_filter_query(contexts, row["zoll"])
filter_query

{'$and': [{'category': 'vehicle rim'},
  {'hersteller': 'MAM'},
  {'design': 'GT1'},
  {'zoll': '18.0'}]}

In [111]:
question = "Lochkreise: 5/108,5/114,3; Einpresstiefe: 45.0; Nabe: 72.6"

context_fields = [
    "felgen_id",
    "felgen_anwendungs_id",
    "hersteller",
    "design",
    "farbe",
    "lochkreise",
    "zoll",
    "breite",
    "einpresstiefe",
    "nabe",
    "traglast",
    "gewicht",
]

final, final_details = map_similar_rims_pinecone(
    question=question,
    index=index,
    filter_query=filter_query,
    engine_name="text-embedding-ada-002",  # You can replace this with your own engine name
    max_len=1200,
    limit=5,
    threshold=0.85,
    context_fields=context_fields,
)

for item in final:
    print(f"{item =}")
print(final)
print(f"{final_details =}")

item ={'felgen_id': '11205', 'felgen_anwendungs_id': '295540', 'hersteller': 'MAM', 'design': 'GT1', 'farbe': 'silver painted', 'lochkreise': '5/108, 5/114,3', 'zoll': '18.0', 'breite': '8.0', 'einpresstiefe': '45.0', 'nabe': '72.6', 'traglast': '690', 'gewicht': '13250'}
item ={'felgen_id': '9093', 'felgen_anwendungs_id': '295545', 'hersteller': 'MAM', 'design': 'GT1', 'farbe': 'matt black lip polish', 'lochkreise': '5/108, 5/114,3', 'zoll': '18.0', 'breite': '8.0', 'einpresstiefe': '45.0', 'nabe': '72.6', 'traglast': '690', 'gewicht': '13250'}
item ={'felgen_id': '18225', 'felgen_anwendungs_id': '459220', 'hersteller': 'MAM', 'design': 'GT1', 'farbe': 'black front polish', 'lochkreise': '5/108, 5/114,3', 'zoll': '18.0', 'breite': '8.0', 'einpresstiefe': '45.0', 'nabe': '72.6', 'traglast': '690', 'gewicht': '13250'}
item ={'felgen_id': '9656', 'felgen_anwendungs_id': '295570', 'hersteller': 'MAM', 'design': 'GT1', 'farbe': 'palladium lip polish', 'lochkreise': '5/100, 5/112', 'zoll': 

In [93]:
row = pd.read_pickle("data/test_row.pkl")

In [130]:
if "rim_details_system" not in row:
    row["rim_details_system"] = final
else:
    row["rim_details_system"] = final

row

requests_index                                                              0
vehicle_flag                                                   97_3_5052979_0
vehicle                                 Volvo V60 Kombi F 2013 - 2018  T5 2.0
hersteller                                                                MAM
design                                                     GT1 silver painted
farbe                                                                  Silver
lochkreise                                                      5/108,5/114,3
zoll                                                                     18.0
breite                                                                    8.0
einpresstiefe                                                            45.0
nabe                                                                     72.6
traglast                                                                  690
gewicht                                                         

In [108]:
for column in columns_to_match:
    value_to_match = row[column]

    if value_to_match != "nan":
        # Check if filtering by the current column results in an empty DataFrame
        temp_filtered = filtered_df[filtered_df[column].str.contains(value_to_match, case=False, na=False)]

        # If the resulting DataFrame isn't empty, use it
        if not temp_filtered.empty:
            filtered_df = temp_filtered

filtered_df

,felgen_id,felgen_anwendungs_id,hersteller,design,farbe,lochkreise,zoll,breite,einpresstiefe,nabe,traglast,gewicht
2,11205,295540,MAM,GT1,silver painted,"5/108, 5/114,3",18.0,8.0,45.0,72.6,690,13250


In [122]:
def filter_rim_details(columns_to_match: List[str], row: Dict[str, any]) -> List[Dict[str, any]]:
    """
    Filter a list of rim details based on specified columns and values.

    Parameters:
        columns_to_match (List[str]): List of columns to filter by.
        row (Dict[str, Any]): Dictionary containing values to match for each specified column.

    Returns:
        List[Dict[str, Any]]: Filtered list of dictionaries containing rim details.
    """
    # Convert the list of dictionaries to a DataFrame
    rim_details = row["rim_details_system"]

    df = pd.DataFrame(rim_details)

    # Iterate through the specified columns to apply filtering
    for column in columns_to_match:
        value_to_match = row.get(column, "nan")

        if value_to_match != "nan":
            temp_filtered = df[df[column].str.contains(str(value_to_match), case=False, na=False)]

            if not temp_filtered.empty:
                df = temp_filtered

    # Convert the filtered DataFrame back to a list of dictionaries
    return df.to_dict("records")

In [129]:
columns_to_match = [
    "farbe",
    "lochkreise",
    "zoll",
    "breite",
    "einpresstiefe",
    "nabe",
]
rim_details = filter_rim_details(columns_to_match, row)
rim_details

[{'felgen_id': '11205',
  'felgen_anwendungs_id': '295540',
  'hersteller': 'MAM',
  'design': 'GT1',
  'farbe': 'silver painted',
  'lochkreise': '5/108, 5/114,3',
  'zoll': '18.0',
  'breite': '8.0',
  'einpresstiefe': '45.0',
  'nabe': '72.6',
  'traglast': '690',
  'gewicht': '13250'}]